In [232]:
import numpy as np
import pandas as pd
from collections import Counter
from crawl_utils.url_extractor import *
from crawl_utils.table_parser import *
from crawl_utils.html_request import *
from crawl_utils.create_table import *
from crawl_utils.html_parser import *
from crawl_utils.pickle_io import *

In [200]:
HSPT_URL = pd.read_csv('HSPT_URL.csv')
HSPT_CHILDREN_URL = pd.read_csv('HSPT_CHILDREN_URL_depth_1.csv')

In [231]:
NON_PAYMENT

,hspt_name,url,depth,text,date
242,연세사랑병원,http://www.yonserang.com/template/etc/non_pay_...,1,연세사랑병원/비급여진료비이용안내,200313
297,연세사랑병원,http://www.yonserang.com/template/etc/non_pay_...,1,연세사랑병원/비급여진료비이용안내,200313
317,연세건우병원,http://www.healthyfriend.co.kr/introduce/be,1,연세건우병원/-비급여,200313
399,연세건우병원,http://www.healthyfriend.co.kr/introduce/be,1,연세건우병원/비급여안내,200313
1092,바른세상병원,https://www.barunsesang.co.kr/guide/unlnsuraLi...,1,바른세상병원/비급여 진료비안내,200313
...,...,...,...,...,...
25185,척사랑병원,http://www.spinelove.com/etc/non-payment-item/,1,척사랑병원/\n비급여항목\n척사랑병원 비급여 진료비용을 고지합니다.\n,200313
25189,척사랑병원,http://www.spinelove.com/etc/non-payment-item/,1,척사랑병원/\n비급여항목\n비급여 진료비용을 고지합니다.\n,200313
25193,척사랑병원,http://www.spinelove.com/etc/non-payment-item/,1,척사랑병원/\n\n\n비급여항목\n비급여 진료비용을 고지합니다.\n\n,200313
25215,세연마취통증의학과의원,http://www.pain119.co.kr/homepage.php?homefile...,1,세연마취통증의학과의원/비급여항목,200313


In [201]:
NON_PAYMENT = select_sub_page_by_query(HSPT_CHILDREN_URL, '비급여')

In [242]:
NON_PAYMENT.to_csv('NON_PAYMENT.csv')

In [202]:
visited = set(HSPT_CHILDREN_URL.url)

In [203]:
len(set(NON_PAYMENT.hspt_name))

164

In [172]:
table_list = {}
no_table = set([])
error = set([])
for idx, row in NON_PAYMENT.iterrows():
    try:
        table = table_parsing(row.url)
        if table:
            table_list["{}/{}".format(row.hspt_name, row.text)] = table
        else:
            no_table.update([row.hspt_name])
    except Exception as e:
        print(row.url, e)

404
Not Found
{'Date': 'Fri, 13 Mar 2020 02:23:32 GMT', 'Server': 'Apache/2.2.3 (CentOS)', 'Content-Length': '313', 'Connection': 'close', 'Content-Type': 'text/html; charset=iso-8859-1'}
https://www.sehunghospital.co.kr/sub/treatment/treat05.html?menu=5
javascript:menu0507(); : Error
3
javascript:menu0507(); : Error
2
javascript:menu0507(); : Error
1
javascript:menu0507(); : Error
failed to download javascript:menu0507();
javascript:menu0507(); : Error
3
javascript:menu0507(); : Error
2
javascript:menu0507(); : Error
1
javascript:menu0507(); : Error
failed to download javascript:menu0507();
http://www.scwoori.com/sub03/sub0306.php invalid literal for int() with base 10: '82 width='
javascript:void window.open('/skin/mon/mon_01.html','비급여항목','width=860px,toolbar=no,location=no,directories=no,status=no,menubar=no,scrollbars=yes,resizable=no'); : Error
3
javascript:void window.open('/skin/mon/mon_01.html','비급여항목','width=860px,toolbar=no,location=no,directories=no,status=no,menubar=no,scr

In [206]:
len(set(_.split('/')[0] for _ in table_list.keys()))

134

In [207]:
len(table_list), len([key for key, value in table_list.items() if value])

(176, 176)

In [208]:
column = ['분류', '구분', '명칭', '코드', '비용', '특이사항', '최저비용', '약제비포함여부', '치료재료대포함여부', '최고비용',
         '중분류', '소분류']
change_column = {c:c for c in column}

counter = Counter([c for table in table_list.values() for t in table for c in t.columns]).most_common()
column = {'최대':'최고비용','최저': '최저비용', '최고':'최고비용', '최소':'최저비용', 
          '비고':'특이사항',  '기타':'특이사항', '단위' : '특이사항',
          '단가': '비용', '금액':'비용', '가격':'비용', '(원)':'비용',
          '수가명칭':'명칭', '내용':'명칭', '비급여명':'명칭', '처방명':'명칭', 
          '항목':'명칭', '품목':'명칭', '수가명':'명칭', '명':'명칭', '품명':'명칭', '기본항목':'명칭',
          '수가':'비용', '일반수가':'비용',
          '약제':'약제비포함여부', '약재':'약제비포함여부', 
          '치료재료':'치료재료대포함여부', '치료대':'치료재료대포함여부', }
change_column.update(column)

for c, v in counter:
    c = str(c)
    for col in column:
        if len(c) == 1 or len(col) == 1:
            continue
        if c in col:
            if c =="":
                continue
            change_column[c] = column[col]
        if col in c:
            change_column[c] = column[col]

change_column = dict(filter(lambda e: len(e[0]) < 10 and len(e[0]) > 1, change_column.items()))

In [209]:
change_column['제증명 내역'] = '명칭'
change_column['기본항목'] = '명칭'
change_column['분류/명칭'] = '명칭'
change_column['수가명'] = '명칭'
change_column['상세 분류'] = '명칭'
change_column['세부항목'] = '명칭'
change_column['수가코드'] = '특이사항'
change_column['수가명칭(한글)'] = '명칭'
change_column['최저부가비율'] = '특이사항'
change_column['최고가비율'] = '특이사항'
change_column['비급여가(원)'] = '비용'
change_column['단위'] = '특이사항'

In [210]:
gubun = ['제증명료 구분','상급병실료 구분','식대료 구분','이송료 구분','주사료 구분','처치수술료 구분', '처치수술료 구분', '약제료 구분']
for gub in gubun:
    change_column[gub] = '명칭'

In [211]:
def rename_column(table, change_column):
    table.columns = list(map(query_re, table.columns))
    change_dict = \
    dict(filter(lambda f: f[0] in list(filter(lambda e: e in change_column, table.columns)), 
                change_column.items()))
    table = table.rename(columns = change_dict)
    return table

In [212]:
def filter_column(table, change_column):
    return table[set(filter(lambda e: e in change_column, table.columns))]

In [213]:
def change_duplicate_column(cols):
    new_col = []
    i = 0
    dups = list(filter(lambda e: e[1]>1, Counter(cols).items()))
    dups = [_[0] for _ in dups]
    for _ in cols:
        if _ in dups:
            if not i:
                new_col += [_]
                i += 1
            else:
                new_col += ['중복컬럼_{}'.format(i)]
                i += 1
        else:
            new_col += [_]
    return new_col

In [214]:
null_hspt_list = set([])
filtered_df_list = []
for hspt, tables in table_list.items():
    for table in tables:
        filtered_df = filter_column(rename_column(table, change_column), change_column)
        filtered_df.columns = change_duplicate_column(filtered_df.columns)
        filtered_df = filtered_df.apply(lambda e: [str(_).strip() for _ in e])
        if not filtered_df.empty:
            filtered_df['병원명'] = hspt.split('/')[0]
            filtered_df_list += [filtered_df]
        else:
            null_hspt_list.update([hspt.split('/')[0]])

In [215]:
final_table = reduce(lambda a, b: pd.concat([a,b], ignore_index=True), filtered_df_list)

In [216]:
# final_table = final_table.applymap(lambda e: np.nan if e=="" else e)
final_table = final_table.dropna(how='all')

In [217]:
len(set(final_table.병원명))

116

In [241]:
pickle_save('change_column', change_column)

In [219]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17105 entries, 0 to 17104
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   약제비포함여부    7518 non-null   object
 1   치료재료대포함여부  5953 non-null   object
 2   구분         8467 non-null   object
 3   코드         9400 non-null   object
 4   비용         14484 non-null  object
 5   분류         8523 non-null   object
 6   최저비용       8899 non-null   object
 7   최고비용       8899 non-null   object
 8   특이사항       9586 non-null   object
 9   명칭         14321 non-null  object
 10  병원명        17105 non-null  object
 11  중분류        2559 non-null   object
 12  중복컬럼_1     2642 non-null   object
 13  소분류        513 non-null    object
 14  중복컬럼_2     1113 non-null   object
 15  중복컬럼_3     1113 non-null   object
 16  중복컬럼_4     870 non-null    object
 17  중복컬럼_5     870 non-null    object
 18  중복컬럼_6     830 non-null    object
 19  중복컬럼_7     744 non-null    object
 20  중복컬럼_8     632 non-null    o

In [220]:
final_table.columns

Index(['약제비포함여부', '치료재료대포함여부', '구분', '코드', '비용', '분류', '최저비용', '최고비용', '특이사항',
       '명칭', '병원명', '중분류', '중복컬럼_1', '소분류', '중복컬럼_2', '중복컬럼_3', '중복컬럼_4',
       '중복컬럼_5', '중복컬럼_6', '중복컬럼_7', '중복컬럼_8'],
      dtype='object')

In [221]:
sort_column = ['병원명', '명칭', '분류', '소분류', '중분류', '중복컬럼_1','중복컬럼_2', '비용', '최저비용', '최고비용', 
               '코드', '구분', '특이사항', '약제비포함여부',  '치료재료대포함여부']

In [222]:
len(set(final_table.병원명))

116

In [223]:
no_name = final_table[final_table["명칭"].isna()]

In [224]:
yes_name = final_table[~final_table["명칭"].isna()]

In [225]:
yes_cost = final_table.dropna(how='all', subset=['최고비용', '최저비용', '비용'])

In [226]:
yes_cost = yes_cost[sort_column]

In [230]:
(len(set(HSPT_URL.hspt_name)), 
        len(set(HSPT_CHILDREN_URL.hspt_name)),
        len(set(NON_PAYMENT.hspt_name)),
        len(set(_.split('/')[0] for _ in table_list.keys())),
        len(set(final_table.병원명)),
        len(set(yes_cost.병원명)))

(240, 220, 164, 134, 116, 109)

In [195]:
yes_cost = yes_cost[sort_column]

In [196]:
import time

In [197]:
time_str = time.strftime('%y%m%d', time.localtime(time.time()))

In [39]:
yes_cost.to_csv('yes_cost_{}.csv'.format(time_str), index=None)

In [342]:
yes_cost[yes_cost["명칭"].isnull()]

,병원명,명칭,명칭_2,분류,소분류,중분류,비용,최저비용,최고비용,코드,구분,특이사항,약제비포함여부,치료재료대포함여부
1000,고도일병원,NaN,NaN,자율신경검사,NaN,검사료,NaN,NaN,"50,000",FY894,NaN,NaN,NaN,NaN
1001,고도일병원,NaN,NaN,하지동맥 경화도검사,NaN,검사료,NaN,NaN,"50,000",EZ868,NaN,NaN,NaN,NaN
1005,고도일병원,NaN,NaN,디퓨젼 스캔,NaN,자기공명영상진단료(MRI),NaN,NaN,"100,000",NaN,NaN,급여인정기준외,NaN,NaN
1007,고도일병원,NaN,NaN,MRI(두경부),NaN,자기공명영상진단료(MRI),NaN,NaN,"450,000",HE403,NaN,급여인정기준외,NaN,NaN
1030,고도일병원,NaN,NaN,MRI (경부혈관),NaN,자기공명영상진단료(MRI),NaN,NaN,"450,000",NaN,NaN,급여인정기준외,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11222,전주우리병원,NaN,NaN,NaN,NaN,NaN,"880,000",NaN,NaN,NaN,NaN,1개,NaN,NaN
11223,전주우리병원,NaN,NaN,NaN,NaN,NaN,"1,550,000",NaN,NaN,NaN,NaN,1개,NaN,NaN
11224,전주우리병원,NaN,NaN,NaN,NaN,NaN,"650,000",NaN,NaN,NaN,NaN,1개,NaN,NaN
11225,전주우리병원,NaN,NaN,NaN,NaN,NaN,"800,000",NaN,NaN,NaN,NaN,1개,NaN,NaN


In [275]:
sum(a.isnull())

403

In [216]:
sort_column = ['병원명', '명칭', '명칭_2', '분류', '소분류', '중분류', '비용', '최저비용', '최고비용', 
               '코드', '구분', '특이사항', '약제비포함여부',  '치료재료대포함여부']

In [125]:
len(set(yes_cost.병원명))

90

In [123]:
no_cost = set(final_table.병원명).difference(set((yes_cost.병원명)))

In [124]:
len(set(yes_name.병원명)), len(set(yes_cost.병원명))

(95, 90)

In [ ]:
def get_non_payment_url(query):
    for i, _ in select_sub_page_by_hspt_name(NON_PAYMENT, query).iterrows():
        return _.url
def print_non_payment_url(query):
for i, _ in select_sub_page_by_hspt_name(NON_PAYMENT, query).iterrows():
    print(_.url)

In [112]:
print(yes_cost[yes_cost[sort_column].병원명 == '휴병원']["명칭"])

1305                      1인실
1306       체지방분석검사(일반)-Inbody
1307    Influenza A.B(간이법)-독감
1308                  족부 압력검사
1309                  동맥경화도검사
                ...          
1619       향후 치료비 추정서(천만원 이상)
1620        일반구급차(기본요금)10Km이내
1621      일반구급차(10Km초과 후 Km당)
1622               구급차 의료인 동행
1623               진료기록영상(CD)
Name: 명칭, Length: 314, dtype: object


In [113]:
len(set(yes_cost.병원명))

90

In [ ]:
for q in list(no_cost):
    print_non_payment_url(q)

In [ ]:
len(set(final_table["병원명"]))

In [ ]:
visited.update(set(NON_PAYMENT.url))
NON_2 = get_sub_pages(NON_PAYMENT, visited)
NON_2_df = get_html_table(NON_2, 2)

In [ ]:
for idx, row in NON_2_df.iterrows():
    table_list["{}/{}".format(row.hspt_name, row.text)] = table_parsing(row.url)

In [ ]:
visited.update(set(NON_2_df.url))
NON_3 = get_sub_pages(NON_2_df, visited)
NON_3_df = get_html_table(NON_3, 3)

In [ ]:
visited.update(set(NON_3_df.url))
NON_4 = get_sub_pages(NON_3_df, visited)
NON_4_df = get_html_table(NON_4, 4)

In [ ]:
visited.update(set(NON_4_df.url))
NON_5 = get_sub_pages(NON_4_df, visited)
NON_5_df = get_html_table(NON_5, 5)

In [ ]:
visited.update(set(NON_5_df.url))
NON_6 = get_sub_pages(NON_5_df, visited)
NON_6_df = get_html_table(NON_6, 6)

In [ ]:
next_signs = ['>', '>>', '다음']

In [ ]:
NON_5_df

In [ ]:
for idx, row in non_2_df.iterrows():
    table_list[row.url] = table_parsing(row.url)

In [ ]:
len(table_list)